# Factory Sensor Data Analysis and Reporting

In [212]:
import pandas as pd

## Dataset

In [213]:
df = pd.read_csv('Data/industrial_sensor_data.csv', parse_dates=['timestamp'])

In [214]:
pd.options.display.max_rows = 10

In [215]:
df['timestamp'] = df['timestamp'].dt.ceil('S')

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\1873756507.py:1: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['timestamp'] = df['timestamp'].dt.ceil('S')


In [216]:
df.head()

,timestamp,machine_id,sensor_type,sensor_value,unit
0,2025-09-27 02:28:45,M07,pressure,3.26,bar
1,2025-09-26 03:13:45,M02,temperature,80.96,C
2,2025-09-19 07:18:45,M09,pressure,4.81,bar
3,2025-09-13 13:18:45,M01,pressure,3.68,bar
4,2025-09-28 20:23:45,M06,temperature,79.97,C


In [217]:
len(df)

259200

## Data Cleaning

In [218]:
df.isnull().sum()

timestamp       0
machine_id      0
sensor_type     0
sensor_value    0
unit            0
dtype: int64

In [219]:
df.duplicated().value_counts()

False    259200
Name: count, dtype: int64

In [220]:
df2 = df.copy()

In [221]:
df2['unit'][df2['unit']=='C'] = 'Celsius'

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\1812997798.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df2['unit'][df2['unit']=='C'] = 'Celsius'
C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\1812997798.py:1: SettingWithCo

In [222]:
df2

,timestamp,machine_id,sensor_type,sensor_value,unit
0,2025-09-27 02:28:45,M07,pressure,3.26,bar
1,2025-09-26 03:13:45,M02,temperature,80.96,Celsius
2,2025-09-19 07:18:45,M09,pressure,4.81,bar
3,2025-09-13 13:18:45,M01,pressure,3.68,bar
4,2025-09-28 20:23:45,M06,temperature,79.97,Celsius
...,...,...,...,...,...
259195,2025-09-14 14:18:45,M07,temperature,71.59,Celsius
259196,2025-09-14 14:23:45,M09,pressure,4.33,bar
259197,2025-09-28 13:18:45,M03,pressure,3.38,bar
259198,2025-09-22 10:53:45,M07,vibration,0.08,mm/s


## Data Transformation

### Pivot the table to have each sensor as a column

In [223]:
df_pivot = df.pivot(index=['timestamp','machine_id'], columns='sensor_type', values='sensor_value').reset_index().rename_axis(None, axis=1)

In [224]:
df_pivot

,timestamp,machine_id,pressure,temperature,vibration
0,2025-08-30 15:43:45,M01,3.91,83.38,0.09
1,2025-08-30 15:43:45,M02,4.12,70.24,0.02
2,2025-08-30 15:43:45,M03,4.29,84.94,0.09
3,2025-08-30 15:43:45,M04,2.20,74.16,0.02
4,2025-08-30 15:43:45,M05,2.21,86.23,0.05
...,...,...,...,...,...
86395,2025-09-29 15:38:45,M06,3.96,73.35,0.05
86396,2025-09-29 15:38:45,M07,4.04,69.75,0.07
86397,2025-09-29 15:38:45,M08,3.45,84.86,0.04
86398,2025-09-29 15:38:45,M09,1.63,81.68,0.10


### Resample data to hourly/daily averages

In [225]:
df3 = df.set_index('timestamp')

In [226]:
df3

,machine_id,sensor_type,sensor_value,unit
timestamp,,,,
2025-09-27 02:28:45,M07,pressure,3.26,bar
2025-09-26 03:13:45,M02,temperature,80.96,C
2025-09-19 07:18:45,M09,pressure,4.81,bar
2025-09-13 13:18:45,M01,pressure,3.68,bar
2025-09-28 20:23:45,M06,temperature,79.97,C
...,...,...,...,...
2025-09-14 14:18:45,M07,temperature,71.59,C
2025-09-14 14:23:45,M09,pressure,4.33,bar
2025-09-28 13:18:45,M03,pressure,3.38,bar


In [227]:
df3.resample('h').sum(numeric_only=True)

,sensor_value
timestamp,
2025-08-30 15:00:00,3290.11
2025-08-30 16:00:00,9565.45
2025-08-30 17:00:00,9742.81
2025-08-30 18:00:00,9757.66
2025-08-30 19:00:00,9626.44
...,...
2025-09-29 11:00:00,9708.62
2025-09-29 12:00:00,9666.14
2025-09-29 13:00:00,9782.27


In [228]:
df3.resample('D').sum(numeric_only=True)

,sensor_value
timestamp,
2025-08-30,80992.76
2025-08-31,232273.64
2025-09-01,232162.20
2025-09-02,231813.99
2025-09-03,231728.61
...,...
2025-09-25,231904.60
2025-09-26,232361.54
2025-09-27,232117.82


In [229]:
df4 = df.groupby(['machine_id','sensor_type',pd.Grouper(key='timestamp', freq='h')]).mean(numeric_only=True).reset_index()

In [230]:
df4

,machine_id,sensor_type,timestamp,sensor_value
0,M01,pressure,2025-08-30 15:00:00,3.02
1,M01,pressure,2025-08-30 16:00:00,3.33
2,M01,pressure,2025-08-30 17:00:00,3.07
3,M01,pressure,2025-08-30 18:00:00,3.55
4,M01,pressure,2025-08-30 19:00:00,2.89
...,...,...,...,...
21625,M10,vibration,2025-09-29 11:00:00,0.05
21626,M10,vibration,2025-09-29 12:00:00,0.06
21627,M10,vibration,2025-09-29 13:00:00,0.04
21628,M10,vibration,2025-09-29 14:00:00,0.05


In [231]:
df5 = df.groupby(['machine_id','sensor_type',pd.Grouper(key='timestamp', freq='D')]).mean(numeric_only=True).reset_index()

In [232]:
df5

,machine_id,sensor_type,timestamp,sensor_value
0,M01,pressure,2025-08-30,3.05
1,M01,pressure,2025-08-31,2.92
2,M01,pressure,2025-09-01,3.00
3,M01,pressure,2025-09-02,2.98
4,M01,pressure,2025-09-03,2.98
...,...,...,...,...
925,M10,vibration,2025-09-25,0.05
926,M10,vibration,2025-09-26,0.05
927,M10,vibration,2025-09-27,0.05
928,M10,vibration,2025-09-28,0.05


### Compute rolling statistics

In [233]:
df6 = df.set_index('timestamp')

In [234]:
df6.sort_index(inplace=True)

In [235]:
df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1h').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\1753753003.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1h').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])


,machine_id,sensor_type,sensor_value,unit,rolling_mean,rolling_std
timestamp,,,,,,
2025-08-30 15:43:45,M01,pressure,3.91,bar,3.91,NaN
2025-08-30 15:48:45,M01,pressure,1.79,bar,2.85,1.50
2025-08-30 15:53:45,M01,pressure,1.80,bar,2.50,1.22
2025-08-30 15:58:45,M01,pressure,4.60,bar,3.02,1.45
2025-08-30 16:03:45,M01,pressure,3.87,bar,3.19,1.31
...,...,...,...,...,...,...
2025-09-29 15:18:45,M10,vibration,0.03,mm/s,0.05,0.02
2025-09-29 15:23:45,M10,vibration,0.04,mm/s,0.05,0.02
2025-09-29 15:28:45,M10,vibration,0.09,mm/s,0.05,0.03


In [236]:
df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1D').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\2746412287.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1D').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])


,machine_id,sensor_type,sensor_value,unit,rolling_mean,rolling_std
timestamp,,,,,,
2025-08-30 15:43:45,M01,pressure,3.91,bar,3.91,NaN
2025-08-30 15:48:45,M01,pressure,1.79,bar,2.85,1.50
2025-08-30 15:53:45,M01,pressure,1.80,bar,2.50,1.22
2025-08-30 15:58:45,M01,pressure,4.60,bar,3.02,1.45
2025-08-30 16:03:45,M01,pressure,3.87,bar,3.19,1.31
...,...,...,...,...,...,...
2025-09-29 15:18:45,M10,vibration,0.03,mm/s,0.05,0.02
2025-09-29 15:23:45,M10,vibration,0.04,mm/s,0.05,0.02
2025-09-29 15:28:45,M10,vibration,0.09,mm/s,0.05,0.03


## Data Analysis

### Detect anomalies: Temperature exceeding thresholds and vibration spikes

In [237]:
df[df['sensor_type']=='temperature'].max()

timestamp       2025-09-29 15:38:45
machine_id                      M10
sensor_type             temperature
sensor_value                  90.00
unit                              C
dtype: object

In [238]:
df[df['sensor_type']=='temperature'].min()

timestamp       2025-08-30 15:43:45
machine_id                      M01
sensor_type             temperature
sensor_value                  65.00
unit                              C
dtype: object

In [239]:
df7 = df[df['sensor_type']=='temperature']

In [240]:
temp_anomaly = df7[(df7['sensor_value']<70) | (df7['sensor_value']>85)]

In [241]:
temp_anomaly

,timestamp,machine_id,sensor_type,sensor_value,unit
21,2025-09-19 12:18:45,M03,temperature,85.60,C
30,2025-09-28 18:13:45,M09,temperature,67.04,C
42,2025-09-27 08:58:45,M09,temperature,69.13,C
43,2025-09-26 03:18:45,M07,temperature,86.41,C
48,2025-09-24 13:03:45,M10,temperature,85.50,C
...,...,...,...,...,...
259151,2025-09-18 11:08:45,M03,temperature,87.86,C
259154,2025-09-05 02:48:45,M07,temperature,66.15,C
259169,2025-09-24 07:08:45,M05,temperature,65.97,C
259171,2025-09-26 11:13:45,M09,temperature,65.49,C


In [242]:
temp_anomaly.sort_values(by=['machine_id','timestamp']).set_index('timestamp')

,machine_id,sensor_type,sensor_value,unit
timestamp,,,,
2025-08-30 16:23:45,M01,temperature,86.58,C
2025-08-30 16:28:45,M01,temperature,66.58,C
2025-08-30 16:48:45,M01,temperature,87.94,C
2025-08-30 16:58:45,M01,temperature,66.43,C
2025-08-30 17:18:45,M01,temperature,86.84,C
...,...,...,...,...
2025-09-29 14:43:45,M10,temperature,86.57,C
2025-09-29 15:03:45,M10,temperature,85.35,C
2025-09-29 15:13:45,M10,temperature,67.35,C


In [243]:
df8 = df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1h').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\3780566630.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df8 = df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1h').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])


In [244]:
vib_anomaly = df8[(df8['sensor_value'] - df8['rolling_mean']).abs() > 2.5*df8['rolling_std']]

In [245]:
vib_anomaly

,machine_id,sensor_type,sensor_value,unit,rolling_mean,rolling_std
timestamp,,,,,,
2025-09-01 06:03:45,M01,pressure,4.26,bar,2.19,0.78
2025-09-26 18:03:45,M01,pressure,4.94,bar,2.75,0.84
2025-09-29 11:58:45,M01,pressure,1.29,bar,3.40,0.84
2025-09-11 14:58:45,M01,temperature,67.18,C,80.48,4.93
2025-09-18 22:18:45,M01,temperature,65.12,C,82.45,6.77
...,...,...,...,...,...,...
2025-09-19 18:48:45,M10,temperature,65.92,C,84.81,7.05
2025-09-05 18:28:45,M10,vibration,0.02,mm/s,0.07,0.02
2025-09-24 01:13:45,M10,vibration,0.09,mm/s,0.04,0.02


In [246]:
temp_anomaly['anomaly'] = 'temperature anomaly'

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\477448550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_anomaly['anomaly'] = 'temperature anomaly'


In [247]:
vib_anomaly['anomaly'] = 'vibration spike'

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\4031240392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vib_anomaly['anomaly'] = 'vibration spike'


In [248]:
anomaly_table = df.merge(temp_anomaly, how='outer')

In [249]:
anomaly_table

,timestamp,machine_id,sensor_type,sensor_value,unit,anomaly
0,2025-08-30 15:43:45,M01,pressure,3.91,bar,NaN
1,2025-08-30 15:43:45,M01,temperature,83.38,C,NaN
2,2025-08-30 15:43:45,M01,vibration,0.09,mm/s,NaN
3,2025-08-30 15:43:45,M02,pressure,4.12,bar,NaN
4,2025-08-30 15:43:45,M02,temperature,70.24,C,NaN
...,...,...,...,...,...,...
259195,2025-09-29 15:38:45,M09,temperature,81.68,C,NaN
259196,2025-09-29 15:38:45,M09,vibration,0.10,mm/s,NaN
259197,2025-09-29 15:38:45,M10,pressure,2.52,bar,NaN
259198,2025-09-29 15:38:45,M10,temperature,79.78,C,NaN


In [250]:
vib_anomaly.reset_index(inplace=True)

In [251]:
vib_anomaly.drop(columns=['rolling_mean','rolling_std'], inplace=True)

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\3819942728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vib_anomaly.drop(columns=['rolling_mean','rolling_std'], inplace=True)


In [252]:
anomaly_table = anomaly_table.merge(vib_anomaly, how='outer')

In [253]:
anomaly_table

,timestamp,machine_id,sensor_type,sensor_value,unit,anomaly
0,2025-08-30 15:43:45,M01,pressure,3.91,bar,NaN
1,2025-08-30 15:43:45,M01,temperature,83.38,C,NaN
2,2025-08-30 15:43:45,M01,vibration,0.09,mm/s,NaN
3,2025-08-30 15:43:45,M02,pressure,4.12,bar,NaN
4,2025-08-30 15:43:45,M02,temperature,70.24,C,NaN
...,...,...,...,...,...,...
259283,2025-09-29 15:38:45,M09,temperature,81.68,C,NaN
259284,2025-09-29 15:38:45,M09,vibration,0.10,mm/s,NaN
259285,2025-09-29 15:38:45,M10,pressure,2.52,bar,NaN
259286,2025-09-29 15:38:45,M10,temperature,79.78,C,NaN


### Identify machines with unusual patterns

In [254]:
anomaly_machines = anomaly_table.groupby(['machine_id','sensor_type']).agg({'anomaly':'count'}).reset_index()

In [255]:
anomaly_machines['machine_id'][anomaly_machines['anomaly']>0].drop_duplicates()

0     M01
4     M02
6     M03
9     M04
12    M05
15    M06
18    M07
22    M08
24    M09
27    M10
Name: machine_id, dtype: object

In [256]:
anomaly_table['z_score'] = anomaly_table.groupby('sensor_type')['sensor_value'].transform(lambda x: (x - x.mean()) / x.std())

In [257]:
anomaly_table

,timestamp,machine_id,sensor_type,sensor_value,unit,anomaly,z_score
0,2025-08-30 15:43:45,M01,pressure,3.91,bar,NaN,0.79
1,2025-08-30 15:43:45,M01,temperature,83.38,C,NaN,0.81
2,2025-08-30 15:43:45,M01,vibration,0.09,mm/s,NaN,1.30
3,2025-08-30 15:43:45,M02,pressure,4.12,bar,NaN,0.97
4,2025-08-30 15:43:45,M02,temperature,70.24,C,NaN,-1.01
...,...,...,...,...,...,...,...
259283,2025-09-29 15:38:45,M09,temperature,81.68,C,NaN,0.58
259284,2025-09-29 15:38:45,M09,vibration,0.10,mm/s,NaN,1.61
259285,2025-09-29 15:38:45,M10,pressure,2.52,bar,NaN,-0.42
259286,2025-09-29 15:38:45,M10,temperature,79.78,C,NaN,0.31


In [258]:
anomaly_table['anomaly'][anomaly_table['z_score'].abs() > 1.7] = 'z_score anomaly'

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\974432146.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  anomaly_table['anomaly'][anomaly_table['z_score'].abs() > 1.7] = 'z_score anomaly'
C:\Users\smazumdar\AppData\Local\Temp\ipyke

In [259]:
anomaly_table['machine_id'][anomaly_table['anomaly']=='z_score anomaly'].drop_duplicates().sort_values()

4383    M01
755     M02
848     M03
789     M04
462     M05
106     M06
169     M07
502     M08
1135    M09
177     M10
Name: machine_id, dtype: object

### Calculate overall equipment efficiency (OEE)

In [260]:
machine_time = pd.read_csv('Data/machine_operation_times.csv')

In [261]:
machine_time.dtypes

machine_id           object
planned_time          int64
downtime              int64
good_units            int64
total_units           int64
ideal_cycle_time    float64
dtype: object

In [262]:
machine_time

,machine_id,planned_time,downtime,good_units,total_units,ideal_cycle_time
0,M04,480,88,294,314,1.00
1,M10,480,50,355,390,1.00
2,M05,480,28,379,402,1.00
3,M05,480,39,343,362,1.00
4,M10,480,100,307,323,1.00
...,...,...,...,...,...,...
45,M10,480,22,398,413,1.00
46,M08,480,3,403,414,1.00
47,M05,480,92,346,371,1.00
48,M09,480,15,395,439,1.00


In [263]:
machine_time['operating_time'] = machine_time['planned_time'] - machine_time['downtime']

In [264]:
machine_time['availability'] = machine_time['operating_time'] / machine_time['planned_time']

In [265]:
machine_time['theoretical_output'] = machine_time['operating_time'] / machine_time['ideal_cycle_time']

In [266]:
machine_time['performance'] = machine_time['good_units'] / machine_time['theoretical_output']

In [267]:
machine_time['quality'] = machine_time['good_units'] / machine_time['total_units']

In [268]:
machine_time['OEE'] = machine_time['availability'] * machine_time['performance'] * machine_time['quality']

In [269]:
machine_time['OEE %'] = (machine_time['OEE'] * 100).round(2)

In [270]:
pd.options.display.float_format = '{:.2f}'.format

In [271]:
machine_time.groupby('machine_id')['OEE %'].mean().reset_index()

,machine_id,OEE %
0,M01,72.86
1,M02,66.12
2,M03,79.18
3,M04,69.09
4,M05,73.18
5,M06,69.09
6,M07,74.53
7,M08,69.09
8,M09,66.47
9,M10,71.40


## Aggregation & Reporting

### Daily summary per machine

In [272]:
anomaly_table.set_index('timestamp', inplace=True)

In [273]:
daily_summary = anomaly_table.groupby('machine_id').resample('D').apply(lambda x: pd.Series({
    'temp_max': x[x['sensor_type']=='temperature']['sensor_value'].max(),
    'temp_min': x[x['sensor_type']=='temperature']['sensor_value'].min(),
    'temp_avg': x[x['sensor_type']=='temperature']['sensor_value'].mean(),
    'vib_max': x[x['sensor_type']=='vibration']['sensor_value'].max(),
    'num_temp_anomalies': (x['anomaly']=='temperature anomaly').sum(),
    'num_vib_anomalies': (x['anomaly']=='vibration spike').sum(),
    'num_zscore_anomalies': (x['anomaly']=='z_score anomaly').sum()
})).reset_index()

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_5840\1802679685.py:1: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_summary = anomaly_table.groupby('machine_id').resample('D').apply(lambda x: pd.Series({


In [274]:
daily_summary

,machine_id,timestamp,temp_max,temp_min,temp_avg,vib_max,num_temp_anomalies,num_vib_anomalies,num_zscore_anomalies
0,M01,2025-08-30,89.52,65.39,78.42,0.10,30.00,0.00,0.00
1,M01,2025-08-31,89.99,65.12,77.42,0.10,111.00,1.00,9.00
2,M01,2025-09-01,89.81,65.03,77.77,0.10,123.00,1.00,11.00
3,M01,2025-09-02,89.89,65.01,77.30,0.10,108.00,0.00,12.00
4,M01,2025-09-03,89.79,65.21,77.56,0.10,117.00,1.00,9.00
...,...,...,...,...,...,...,...,...,...
305,M10,2025-09-25,89.98,65.23,78.15,0.10,113.00,0.00,11.00
306,M10,2025-09-26,89.99,65.03,78.11,0.10,93.00,1.00,20.00
307,M10,2025-09-27,89.92,65.01,77.54,0.10,97.00,1.00,11.00
308,M10,2025-09-28,89.99,65.06,76.73,0.10,121.00,0.00,18.00


### Generate CSV reports

In [276]:
anomaly_table.to_csv('Data/anomaly_report.csv')

In [277]:
daily_summary.to_csv('Data/daily_machine_summary.csv', index=False)